# TODO: Title

This notebook lists all the steps that you need to complete the complete this project. You will need to complete all the TODOs in this notebook as well as in the README and the two python scripts included with the starter code.


**TODO**: Give a helpful introduction to what this notebook is for. Remember that comments, explanations and good documentation make your project informative and professional.

**Note:** This notebook has a bunch of code and markdown cells with TODOs that you have to complete. These are meant to be helpful guidelines for you to finish your project while meeting the requirements in the project rubrics. Feel free to change the order of these the TODO's and use more than one TODO code cell to do all your tasks.

In [6]:
# TODO: Install any packages that you might need
# For instance, you will need the smdebug package
!pip install smdebug

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 44.5 MB/s  0:00:00
  DEPRECATION: Building 'pyinstrument-cext' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'pyinstrument-cext'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for pyinstrument-cext: filename=pyinstrument_cext-0.2.4-cp310-cp310-linux_x86_64.whl size=6353 sha256=a7e85114c789df9cd0b96e681e5d9f7a7e9428c7ffc62676ed6a63de67386156
  Stored in directory: /home/ec2-user/.cache/pip/wheels/0f/8b/7a/5f7fd1dd6d3cbb3d350d4c832c5e2f962687749f6d67d573a6
Successfully built pyinstrument-cext
  Attempting uninstall: protobuf
    Found existing installation: proto

In [3]:
# TODO: Import any packages that you might need
# For instance you will need Boto3 and Sagemaker
import sagemaker
from sagemaker.pytorch import PyTorch
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter
from sagemaker.tuner import HyperparameterTuner
from sagemaker.debugger import Rule, ProfilerRule, rule_configs
from sagemaker.debugger import DebuggerHookConfig, ProfilerConfig, FrameworkProfile, CollectionConfig
import boto3

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


## Dataset
TODO: Explain what dataset you are using for this project. Maybe even give a small overview of the classes, class distributions etc that can help anyone not familiar with the dataset get a better understand of it.

In [ ]:
#TODO: Fetch and upload the data to AWS S3

# Command to download and unzip data
!wget https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip
!unzip dogImages.zip

In [4]:
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket="sagemaker-studio-869935066996-fi5k7y1ilzt"


In [ ]:
# upload data to S3

# s3 = boto3.client('s3')

# Upload training data
# !aws s3 cp dogImages/ s3://{bucket}/dogImages/ --recursive
# OR use boto3 programmatically

!aws s3 sync dogImages/train  s3://sagemaker-studio-869935066996-fi5k7y1ilzt/train  --dryrun
!aws s3 sync dogImages/valid    s3://sagemaker-studio-869935066996-fi5k7y1ilzt/val    --dryrun
!aws s3 sync dogImages/test   s3://sagemaker-studio-869935066996-fi5k7y1ilzt/test   --dryrun


In [30]:
!aws s3 sync dogImages/train  s3://sagemaker-studio-869935066996-fi5k7y1ilzt/train  --exact-timestamps
!aws s3 sync dogImages/valid    s3://sagemaker-studio-869935066996-fi5k7y1ilzt/val    --exact-timestamps
!aws s3 sync dogImages/test   s3://sagemaker-studio-869935066996-fi5k7y1ilzt/test   --exact-timestamps


upload: dogImages/valid/001.Affenpinscher/Affenpinscher_00038.jpg to s3://sagemaker-studio-869935066996-fi5k7y1ilzt/val/001.Affenpinscher/Affenpinscher_00038.jpg
upload: dogImages/valid/001.Affenpinscher/Affenpinscher_00041.jpg to s3://sagemaker-studio-869935066996-fi5k7y1ilzt/val/001.Affenpinscher/Affenpinscher_00041.jpg
upload: dogImages/valid/001.Affenpinscher/Affenpinscher_00068.jpg to s3://sagemaker-studio-869935066996-fi5k7y1ilzt/val/001.Affenpinscher/Affenpinscher_00068.jpg
upload: dogImages/valid/001.Affenpinscher/Affenpinscher_00049.jpg to s3://sagemaker-studio-869935066996-fi5k7y1ilzt/val/001.Affenpinscher/Affenpinscher_00049.jpg
upload: dogImages/valid/002.Afghan_hound/Afghan_hound_00115.jpg to s3://sagemaker-studio-869935066996-fi5k7y1ilzt/val/002.Afghan_hound/Afghan_hound_00115.jpg
upload: dogImages/valid/001.Affenpinscher/Affenpinscher_00040.jpg to s3://sagemaker-studio-869935066996-fi5k7y1ilzt/val/001.Affenpinscher/Affenpinscher_00040.jpg
upload: dogImages/valid/001.Affe

## Hyperparameter Tuning
**TODO:** This is the part where you will finetune a pretrained model with hyperparameter tuning. Remember that you have to tune a minimum of two hyperparameters. However you are encouraged to tune more. You are also encouraged to explain why you chose to tune those particular hyperparameters and the ranges.

**Note:** You will need to use the `hpo.py` script to perform hyperparameter tuning.

In [5]:
#TODO: Declare your HP ranges, metrics etc.
hyperparameter_ranges = {
    'lr': ContinuousParameter(0.001, 0.1),        # Learning rate from 0.001 to 0.1
    'batch_size': CategoricalParameter([16, 32, 64, 128]),  # Different batch sizes to try
    'epochs': IntegerParameter(2, 5),              # Number of training epochs
}

# Define what metric we want to optimize
# This tells SageMaker what to look for in the logs
objective_metric_name = 'Testing Accuracy'
objective_type = 'Maximize'  # We want to maximize accuracy
metric_definitions = [
    {
        'Name': 'Testing Accuracy',
        'Regex': 'Testing Accuracy: ([0-9\\.]+)'  # This pattern finds accuracy in logs
    },
    {
        'Name': 'Testing Loss', 
        'Regex': 'Testing Loss: ([0-9\\.]+)'      # This pattern finds loss in logs
    }
]


In [6]:
#TODO: Create estimators for your HPs

# TODO: Your estimator here
 
estimator = PyTorch(
    entry_point='hpo.py',                    # Our training script
    source_dir='.',                          # Where to find the script
    role=role,                               # AWS role for permissions
    framework_version='1.8.0',               # PyTorch version
    py_version='py36',                        # Python version
    instance_count=1,                        # How many machines to use
    instance_type='ml.m5.xlarge',            # Type of machine
    hyperparameters={                        # Default hyperparameters
        'epochs': 2,
        'batch_size': 32,
        'lr': 0.01
    },
    metric_definitions=metric_definitions,    # Tell SageMaker what metrics to track
    max_run=3600,                            # Maximum time for one training job (1 hour)
)

# TODO: Your HP tuner here
# Create the hyperparameter tuner
# This is the "smart searcher" that tries different combinations
tuner = HyperparameterTuner(
    estimator=estimator,                     # The estimator template to use
    objective_metric_name=objective_metric_name,  # What to optimize
    objective_type=objective_type,           # Maximize or minimize
    hyperparameter_ranges=hyperparameter_ranges,  # What values to try
    metric_definitions=metric_definitions,   # How to read the metrics
    max_jobs=6,                             # Total number of training jobs to run
    max_parallel_jobs=2,                    # How many to run at the same time
    base_tuning_job_name='dog-breed-hpo'   # Name prefix for the jobs
)

print("Hyperparameter tuner created successfully!")
print(f"Will run {tuner.max_jobs} jobs, {tuner.max_parallel_jobs} at a time")


Hyperparameter tuner created successfully!
Will run 6 jobs, 2 at a time


In [ ]:
# TODO: Fit your HP Tuner

# Define where your training data is stored in S3

data_path = f's3://{bucket}/'
testing_input_path = f's3://{bucket}/test/'
# s3://sagemaker-studio-869935066996-fi5k7y1ilzt/test/
# Create input data configuration

inputs = {
    'data': data_path,
    
}

tuner.fit(inputs) # TODO: Remember to include your data channels
print("Hyperparameter tuning job submitted!")
print(f"Job name: {tuner.latest_tuning_job.job_name}")
print("You can monitor progress in the SageMaker console")




No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


..........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [ ]:
# TODO: Get the best estimators and the best HPs
# Wait for the tuning job to complete (this might take a while)
tuner.wait()

# Get the best training job
best_estimator = tuner.best_estimator()

#Get the hyperparameters of the best trained model
best_estimator.hyperparameters()
print("Best hyperparameters found:")
print(best_estimator.hyperparameters())

## Model Profiling and Debugging
TODO: Using the best hyperparameters, create and finetune a new model

**Note:** You will need to use the `train_model.py` script to perform model profiling and debugging.

In [ ]:
# TODO: Set up debugging and profiling rules and hooks
# These are like "health checks" that run automatically during training
debug_rules = [
    # Check if loss is not decreasing (model might not be learning)
    Rule.sagemaker(rule_configs.loss_not_decreasing()),
    
    # Check if gradients are too small (vanishing gradient problem)
    Rule.sagemaker(rule_configs.vanishing_gradient()),
    
    # Check if gradients are too large (exploding gradient problem)
    Rule.sagemaker(rule_configs.exploding_tensor()),
    
    # Check for overfitting (model memorizing instead of learning)
    Rule.sagemaker(rule_configs.overfit()),
    
    # Check if training loss is not improving
    # Rule.sagemaker(rule_configs.stagnant_training_rule()),
]

# Set up profiling rules
# These monitor resource usage (CPU, GPU, memory)
profiler_rules = [
    # Check if CPU is underutilized
    ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
    
    # Check if GPU memory is efficiently used
    ProfilerRule.sagemaker(rule_configs.GPUMemoryIncrease()),
    
    # Check for inefficient data loading
    ProfilerRule.sagemaker(rule_configs.IOBottleneck()),
]

# Configure what debugging information to collect
debugger_hook_config = DebuggerHookConfig(
    s3_output_path=f's3://{bucket}/debug-output/',
    # collection_configs=[
    #     CollectionConfig(name="weights",   parameters={"save_interval": "100"}),
    #     CollectionConfig(name="gradients", parameters={"save_interval": "10"}),
    #     CollectionConfig(name="losses",    parameters={"save_interval": "1"}),
    #     CollectionConfig(name="metrics",   parameters={"save_interval": "10"}),
    # ]
)

# Configure profiling (resource monitoring)
profiler_config = ProfilerConfig(
    system_monitor_interval_millis=500,  # Monitor system every 500ms
    framework_profile_params=FrameworkProfile(
        local_path="/opt/ml/output/profiler/",  # Local path for profiler data
        start_step=5,      # Start profiling after 5 training steps
        num_steps=10       # Profile for 10 steps
    )
)

print("Debugging and profiling rules configured!")
print(f"Number of debug rules: {len(debug_rules)}")
print(f"Number of profiler rules: {len(profiler_rules)}")

In [ ]:
# TODO: Create and fit an estimator

 # Extract actual best hyperparameters from tuning job
best_hp = tuner.best_estimator().hyperparameters()

# Convert to proper format (removing SageMaker internal parameters)
best_hyperparameters = {
    'lr': float(best_hp['lr']),
    'batch_size': int(best_hp['batch_size'].strip('"')),  # Remove quotes if present
    'epochs': int(best_hp['epochs'])
}


# Get the best hyperparameters from your previous tuning job
# Replace these with your actual best hyperparameters
# best_hyperparameters = {
#     'lr': 0.0015414467052158013,        # Replace with your best learning rate
#     'batch_size': 16,  # Replace with your best batch size  
#     'epochs': 5        # Replace with your best epoch count
# }

print("Using best hyperparameters from tuning:")
for param, value in best_hyperparameters.items():
    print(f"  {param}: {value}")


# Create estimator with debugging and profiling enabled
estimator = PyTorch(
    entry_point='train_model.py',           # Our training script with debug hooks
    source_dir='.',                         # Source directory
    role=role,                              # AWS role
    framework_version='1.8.0',              # PyTorch version
    py_version='py3',                       # Python version
    instance_count=1,                       # Number of instances
    instance_type='ml.p3.2xlarge',          # Use GPU instance for better profiling
    hyperparameters=best_hyperparameters,   # Best hyperparameters from tuning
    
    # Debugging configuration
    rules=debug_rules,                      # Automatic debugging rules
    debugger_hook_config=debugger_hook_config,  # What to collect
    
    # Profiling configuration  
    profiler_rules=profiler_rules,          # Automatic profiling rules
    profiler_config=profiler_config,        # Profiling settings
    
    max_run=7200,                          # Maximum run time (2 hours)
    base_job_name='dog-breed-debug-profile01' # Job name prefix
)

print("Estimator created with debugging and profiling enabled!")


 


In [ ]:
# FIT AN ESTIMATOR

# Define training data location
data_path = f's3://{bucket}/'
 
# Create input data configuration
inputs = {
    'data': data_path,
    
}

 

print("Starting training with debugging and profiling...")
print("This will monitor your model for issues and resource usage!")

# Start training
estimator.fit(inputs, wait=True)

print("Training completed!")

In [ ]:
# TODO: Plot a debugging output.
# Import plotting libraries
import matplotlib.pyplot as plt
from smdebug.trials import create_trial

# Create a trial object to analyze the debugging data
trial = create_trial(estimator.latest_job_debugger_artifacts_path())

print("Available tensor names in debug data:")
print(trial.tensor_names())

# Plot training loss over time
plt.figure(figsize=(12, 8))

# Get loss values
try:
    loss_tensor = trial.tensor('CrossEntropyLoss_output_0')
    steps = loss_tensor.steps()
    loss_values = [loss_tensor.value(s) for s in steps]
    
    plt.subplot(2, 2, 1)
    plt.plot(steps, loss_values)
    plt.title('Training Loss Over Time')
    plt.xlabel('Step')
    plt.ylabel('Loss')
    plt.grid(True)
    
except Exception as e:
    print(f"Could not plot loss: {e}")

# Plot gradient norms (to check for vanishing/exploding gradients)
try:
    # Get gradient tensor for the final layer
    grad_tensor = trial.tensor('fc.weight_grad')
    steps = grad_tensor.steps()
    
    # Calculate gradient norms
    grad_norms = []
    for step in steps:
        grad_value = grad_tensor.value(step)
        grad_norm = np.linalg.norm(grad_value.flatten())
        grad_norms.append(grad_norm)
    
    plt.subplot(2, 2, 2)
    plt.plot(steps, grad_norms)
    plt.title('Gradient Norms Over Time')
    plt.xlabel('Step')
    plt.ylabel('Gradient Norm')
    plt.grid(True)
    
except Exception as e:
    print(f"Could not plot gradients: {e}")

# Plot weight changes
try:
    weight_tensor = trial.tensor('fc.weight')
    steps = weight_tensor.steps()
    
    # Calculate weight norms
    weight_norms = []
    for step in steps:
        weight_value = weight_tensor.value(step)
        weight_norm = np.linalg.norm(weight_value.flatten())
        weight_norms.append(weight_norm)
    
    plt.subplot(2, 2, 3)
    plt.plot(steps, weight_norms)
    plt.title('Weight Norms Over Time')
    plt.xlabel('Step')
    plt.ylabel('Weight Norm')
    plt.grid(True)
    
except Exception as e:
    print(f"Could not plot weights: {e}")

plt.tight_layout()
plt.savefig('debugging_plots.png', dpi=300, bbox_inches='tight')
plt.show()

print("Debugging plots saved as 'debugging_plots.png'")

**TODO**: Is there some anomalous behaviour in your debugging output? If so, what is the error and how will you fix it?  
**TODO**: If not, suppose there was an error. What would that error look like and how would you have fixed it?

In [ ]:
# TODO: Display the profiler output

# Download and display profiler report
profiler_report_name = estimator.latest_job_profiler_artifacts_path() + "/profiler-output/profiler-report.html"

print("Profiler report location:")
print(profiler_report_name)

# You can also check rule evaluation results
print("\nDebugging Rule Evaluation Results:")
for rule in debug_rules:
    print(f"Rule: {rule.name}")
    print(f"Status: {estimator.describe_training_job()['RuleEvaluationStatuses']}")

print("\nProfiling Rule Evaluation Results:")  
for rule in profiler_rules:
    print(f"Rule: {rule.name}")
    
# Display system resource usage
try:
    from smdebug.profiler.analysis.notebook_utils.profiler_report_analysis import ProfilerReportAnalysis
    
    profiler_analysis = ProfilerReportAnalysis(estimator.latest_job_profiler_artifacts_path())
    
    # Display CPU utilization
    profiler_analysis.display_charts(['CPUUtilization'])
    
    # Display GPU utilization (if using GPU instance)
    profiler_analysis.display_charts(['GPUUtilization', 'GPUMemoryUtilization'])
    
except Exception as e:
    print(f"Could not display profiler analysis: {e}")
    print("You can manually download and view the profiler report HTML file")


## Model Deploying

In [ ]:
# TODO: Deploy your model to an endpoint

predictor=estimator.deploy() # TODO: Add your deployment configuration like instance type and number of instances
predictor = estimator.deploy(
    initial_instance_count=1,          # Number of instances (start with 1)
    instance_type='ml.m5.large',       # Instance type (CPU is fine for inference)
    endpoint_name='dog-breed-endpoint', # Give your endpoint a name
    wait=True                          # Wait for deployment to complete
)
print(f"Model deployed successfully!")
print(f"Endpoint name: {predictor.endpoint_name}")

In [1]:
# TODO: Run an prediction on the endpoint

# TODO: Your code to load and preprocess image to send to endpoint for prediction
# Load the image
image = Image.open("test_dog.jpg").convert('RGB')

# Apply the same preprocessing used during training
transform = transforms.Compose([
    transforms.Resize(256),                    # Resize to 256x256
    transforms.CenterCrop(224),                # Crop center 224x224
    transforms.ToTensor(),                     # Convert to tensor
    transforms.Normalize(                      # Normalize like ImageNet
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    )
])

# Preprocess the image
image_tensor = transform(image)
image_tensor = image_tensor.unsqueeze(0)  # Add batch dimension

# Convert to numpy array for sending to endpoint
image_array = image_tensor.numpy()

print(f"Image preprocessed! Shape: {image_array.shape}")

# Send image to endpoint for prediction
response = predictor.predict(image_array)

print("Prediction received!")
print(f"Raw response shape: {response.shape}")
# Process the response to get readable results
probabilities = torch.nn.functional.softmax(torch.from_numpy(response), dim=1)
top5_prob, top5_indices = torch.topk(probabilities, 5)

print("\nPrediction Results:")
print(f"Top predicted class: {top5_indices[0][0].item()}")
print(f"Confidence: {top5_prob[0][0].item():.4f} ({top5_prob[0][0].item()*100:.2f}%)")

print("\nTop 5 Predictions:")
for i in range(5):
    class_id = top5_indices[0][i].item()
    confidence = top5_prob[0][i].item()
    print(f"{i+1}. Class {class_id}: {confidence:.4f} ({confidence*100:.2f}%)")


# response = predictor.predict(image)

NameError: name 'Image' is not defined

In [ ]:
# TODO: Remember to shutdown/delete your endpoint once your work is done
predictor.delete_endpoint()